### Face Recognition with face_recognition library

In [ ]:
# Face Recognition with face_recognition library

import face_recognition
import cv2
import numpy as np
obama_image = face_recognition.load_image_file('Learning/obama.jpg')
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

sachin_image = face_recognition.load_image_file('Learning/sachin.jpg')
sachin_face_encoding = face_recognition.face_encodings(sachin_image)[0]

# Capture Video
cap = cv2.VideoCapture(0)

cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

# Create arrays of known face encodings and their names
known_face_encodings = [
    obama_face_encoding,
    sachin_face_encoding,
]

known_face_names = [
    'Obama',
    'Sachin',
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    ret, frame = cap.read()
    
    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    rgb_small_frame = small_frame[:, :, ::-1]
    
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            
            # use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            
            face_names.append(name)
            
    process_this_frame = not process_this_frame
    
    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        
        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
    
    # Display the resulting image
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

### Attendance with Face Recognition

In [10]:
# Attendance with Face Recognition
# This program will read the images in the listed folder and generate encoding. 
# Read the stream from webcam (id=0) and identify the faces in the webcam stream
# Encode the faces in the frame and compare the encoding with known encodings
# Once match found,  write down the name and time to a csv file

import face_recognition
import cv2
import numpy as np
import os
from datetime import datetime


def findEncodings(images):
    encodeList = []
    
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    
    return encodeList

def markAttendance(name):
    with open('Learning/attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            
        if name not in nameList:
            now = datetime.now()
            dtrString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtrString}')
    

path = 'Learning/faces/'
images = []
classNames = []

myList = os.listdir(path)
#print(myList)

for cl in myList:
    currentImage = cv2.imread(f'{path}/{cl}')
    images.append(currentImage)
    classNames.append(os.path.splitext(cl)[0])
    
#print(classNames)

encodeKnownList = findEncodings(images)
#print('Encoding Completed')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    frame_small = cv2.resize(frame, (0, 0), None, fx=0.25, fy=0.25)
    frame_small_rgb = cv2.cvtColor(frame_small, cv2.COLOR_BGR2RGB)
    
    currentFrameFaces = face_recognition.face_locations(frame_small_rgb)
    currentFrameEncodings = face_recognition.face_encodings(frame_small_rgb, currentFrameFaces)
    
    for frameEncode, faceLoc  in zip(currentFrameEncodings, currentFrameFaces):
        matches = face_recognition.compare_faces(encodeKnownList, frameEncode)
        faceDistance = face_recognition.face_distance(encodeKnownList, frameEncode)
        #print(faceDistance)
        matchIndex = np.argmin(faceDistance)
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            #print(name)
            
            # top, right, bottom, left
            y1, x2, y2, x1 = faceLoc
            
            x1 = x1 * 4
            x2 = x2 * 4
            y1 = y1 * 4
            y2 = y2 * 4
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(frame, (x1, y2-35), (x2, y2), (0, 255, 0), cv2.FILLED)
            font = cv2.FONT_HERSHEY_COMPLEX
            cv2.putText(frame, name, (x1+6, y2-6),font, 1, (255, 255, 255), 2)
            markAttendance(name)

        
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
    
cap.release()
cv2.destroyAllWindows()